#Importing libraries

In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
print(tf.__version__)

2.19.0


#Loading dataset

In [4]:
from pandas.core import indexing
column_names = ['ID','Age','Gender','Height_cm','Weight_kg',
               'BMI','Smoker','Exercise_Freq','Diet_Quality',
               'Alcohol_Consumption','Chronic_Disease','Sleep_Hours']
raw_dataset = pd.read_csv('/content/firstmodel_dataset.csv', names=column_names,
                      na_values = "?", comment='\t',
                      sep=",",index_col=False,skiprows=1)
dataset = raw_dataset.copy()
dataset.tail()

/tmp/ipython-input-2858571989.py:5: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  raw_dataset = pd.read_csv('/content/firstmodel_dataset.csv', names=column_names,


,ID,Age,Gender,Height_cm,Weight_kg,BMI,Smoker,Exercise_Freq,Diet_Quality,Alcohol_Consumption,Chronic_Disease,Sleep_Hours
7495,7496,55,Other,168.3,52.4,18.5,Yes,1-2 times/week,Average,Moderate,Yes,4
7496,7497,24,Male,179.1,58.8,18.3,No,3-5 times/week,Excellent,Moderate,No,5
7497,7498,61,Other,160.2,80.0,31.2,No,3-5 times/week,Poor,Moderate,No,5
7498,7499,40,Female,172.6,66.1,22.2,No,1-2 times/week,Good,Low,No,9
7499,7500,58,Other,163.8,59.3,22.1,No,Daily,Excellent,Low,No,6


#Missing value count

In [5]:
dataset.isna().sum()

,0
ID,0
Age,0
Gender,0
Height_cm,0
Weight_kg,0
BMI,0
Smoker,0
Exercise_Freq,1879
Diet_Quality,0
Alcohol_Consumption,1892


#Drop Missing Value

In [6]:
dataset = dataset.dropna()
dataset

,ID,Age,Gender,Height_cm,Weight_kg,BMI,Smoker,Exercise_Freq,Diet_Quality,Alcohol_Consumption,Chronic_Disease,Sleep_Hours
1,2,69,Other,169.3,70.7,24.7,No,1-2 times/week,Good,High,No,2
2,3,46,Female,159.1,69.0,27.3,No,Daily,Excellent,Moderate,No,3
3,4,32,Male,170.6,76.4,26.3,No,3-5 times/week,Excellent,Moderate,No,9
4,5,60,Male,158.4,60.4,24.1,No,3-5 times/week,Excellent,Low,Yes,6
5,6,25,Female,167.7,62.0,22.0,No,Daily,Good,Moderate,No,1
...,...,...,...,...,...,...,...,...,...,...,...,...
7495,7496,55,Other,168.3,52.4,18.5,Yes,1-2 times/week,Average,Moderate,Yes,4
7496,7497,24,Male,179.1,58.8,18.3,No,3-5 times/week,Excellent,Moderate,No,5
7497,7498,61,Other,160.2,80.0,31.2,No,3-5 times/week,Poor,Moderate,No,5
7498,7499,40,Female,172.6,66.1,22.2,No,1-2 times/week,Good,Low,No,9


#ONE HOT Gender

In [7]:

gender = dataset.pop('Gender')
dataset["Gender_Male"] = (gender == "Male") * 1.0
dataset["Gender_Other"] = (gender == "Other") * 1.0
dataset.tail()

,ID,Age,Height_cm,Weight_kg,BMI,Smoker,Exercise_Freq,Diet_Quality,Alcohol_Consumption,Chronic_Disease,Sleep_Hours,Gender_Male,Gender_Other
7495,7496,55,168.3,52.4,18.5,Yes,1-2 times/week,Average,Moderate,Yes,4,0.0,1.0
7496,7497,24,179.1,58.8,18.3,No,3-5 times/week,Excellent,Moderate,No,5,1.0,0.0
7497,7498,61,160.2,80.0,31.2,No,3-5 times/week,Poor,Moderate,No,5,0.0,1.0
7498,7499,40,172.6,66.1,22.2,No,1-2 times/week,Good,Low,No,9,0.0,0.0
7499,7500,58,163.8,59.3,22.1,No,Daily,Excellent,Low,No,6,0.0,1.0


#ONE HOT Smoker


In [8]:

smoker = dataset.pop("Smoker")
dataset["IsSmoker"] = (smoker=='Yes')*1.0;
dataset.tail()

,ID,Age,Height_cm,Weight_kg,BMI,Exercise_Freq,Diet_Quality,Alcohol_Consumption,Chronic_Disease,Sleep_Hours,Gender_Male,Gender_Other,IsSmoker
7495,7496,55,168.3,52.4,18.5,1-2 times/week,Average,Moderate,Yes,4,0.0,1.0,1.0
7496,7497,24,179.1,58.8,18.3,3-5 times/week,Excellent,Moderate,No,5,1.0,0.0,0.0
7497,7498,61,160.2,80.0,31.2,3-5 times/week,Poor,Moderate,No,5,0.0,1.0,0.0
7498,7499,40,172.6,66.1,22.2,1-2 times/week,Good,Low,No,9,0.0,0.0,0.0
7499,7500,58,163.8,59.3,22.1,Daily,Excellent,Low,No,6,0.0,1.0,0.0


#ONE HOT Alcohol

In [9]:

alcohol = dataset.pop("Alcohol_Consumption")
dataset["Alcohol_Low"] = (alcohol == "Low") * 1.0
dataset["Alcohol_Moderate"] = (alcohol == "Moderate") * 1.0
dataset["Alcohol_High"] = (alcohol == "High") * 1.0
dataset.tail()

,ID,Age,Height_cm,Weight_kg,BMI,Exercise_Freq,Diet_Quality,Chronic_Disease,Sleep_Hours,Gender_Male,Gender_Other,IsSmoker,Alcohol_Low,Alcohol_Moderate,Alcohol_High
7495,7496,55,168.3,52.4,18.5,1-2 times/week,Average,Yes,4,0.0,1.0,1.0,0.0,1.0,0.0
7496,7497,24,179.1,58.8,18.3,3-5 times/week,Excellent,No,5,1.0,0.0,0.0,0.0,1.0,0.0
7497,7498,61,160.2,80.0,31.2,3-5 times/week,Poor,No,5,0.0,1.0,0.0,0.0,1.0,0.0
7498,7499,40,172.6,66.1,22.2,1-2 times/week,Good,No,9,0.0,0.0,0.0,1.0,0.0,0.0
7499,7500,58,163.8,59.3,22.1,Daily,Excellent,No,6,0.0,1.0,0.0,1.0,0.0,0.0


#ONE HOT Diet

In [10]:
diet = dataset.pop("Diet_Quality")
dataset["Diet_Average"] = (diet == "Average") * 1.0
dataset["Diet_Good"] = (diet == "Good") * 1.0
dataset["Diet_Excellent"] = (diet == "Excellent") * 1.0
dataset.tail()

,ID,Age,Height_cm,Weight_kg,BMI,Exercise_Freq,Chronic_Disease,Sleep_Hours,Gender_Male,Gender_Other,IsSmoker,Alcohol_Low,Alcohol_Moderate,Alcohol_High,Diet_Average,Diet_Good,Diet_Excellent
7495,7496,55,168.3,52.4,18.5,1-2 times/week,Yes,4,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
7496,7497,24,179.1,58.8,18.3,3-5 times/week,No,5,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
7497,7498,61,160.2,80.0,31.2,3-5 times/week,No,5,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7498,7499,40,172.6,66.1,22.2,1-2 times/week,No,9,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
7499,7500,58,163.8,59.3,22.1,Daily,No,6,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


#ONE HOT Exercise

In [11]:
exercise = dataset.pop("Exercise_Freq")
dataset["Exercise_1_2"] = (exercise == "1-2 times/week")*1.0
dataset["Exercise_3_5"] = (exercise == "3-5 times/week")*1.0
dataset["Exercise_Daily"] = (exercise == "Daily")*1.0
dataset.tail()

,ID,Age,Height_cm,Weight_kg,BMI,Chronic_Disease,Sleep_Hours,Gender_Male,Gender_Other,IsSmoker,Alcohol_Low,Alcohol_Moderate,Alcohol_High,Diet_Average,Diet_Good,Diet_Excellent,Exercise_1_2,Exercise_3_5,Exercise_Daily
7495,7496,55,168.3,52.4,18.5,Yes,4,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
7496,7497,24,179.1,58.8,18.3,No,5,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
7497,7498,61,160.2,80.0,31.2,No,5,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7498,7499,40,172.6,66.1,22.2,No,9,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
7499,7500,58,163.8,59.3,22.1,No,6,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [12]:

stress = dataset.pop("Chronic_Disease")
dataset["SChronic_Disease_Yes"] = (stress == "Yes") * 1.0
dataset["Chronic_Disease_No"] = (stress == "No") * 1.0
dataset.tail()


,ID,Age,Height_cm,Weight_kg,BMI,Sleep_Hours,Gender_Male,Gender_Other,IsSmoker,Alcohol_Low,Alcohol_Moderate,Alcohol_High,Diet_Average,Diet_Good,Diet_Excellent,Exercise_1_2,Exercise_3_5,Exercise_Daily,SChronic_Disease_Yes,Chronic_Disease_No
7495,7496,55,168.3,52.4,18.5,4,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
7496,7497,24,179.1,58.8,18.3,5,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
7497,7498,61,160.2,80.0,31.2,5,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
7498,7499,40,172.6,66.1,22.2,9,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
7499,7500,58,163.8,59.3,22.1,6,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0


#Dividing Into Training And Testing DataSets

In [13]:
train_dataset = dataset.sample(frac=0.8,random_state=0) #80% data for traing
test_dataset = dataset.drop(train_dataset.index) #testing k liye wo 80% drop kr rahe hai

In [14]:
train_stats = train_dataset.describe()
train_stats

,ID,Age,Height_cm,Weight_kg,BMI,Sleep_Hours,Gender_Male,Gender_Other,IsSmoker,Alcohol_Low,Alcohol_Moderate,Alcohol_High,Diet_Average,Diet_Good,Diet_Excellent,Exercise_1_2,Exercise_3_5,Exercise_Daily,SChronic_Disease_Yes,Chronic_Disease_No
count,3358.000000,3358.000000,3358.000000,3358.000000,3358.000000,3358.00000,3358.000000,3358.000000,3358.000000,3358.000000,3358.000000,3358.000000,3358.000000,3358.000000,3358.000000,3358.00000,3358.000000,3358.000000,3358.000000,3358.000000
mean,3746.601251,43.716200,164.977725,70.148243,26.064116,5.61763,0.339488,0.322811,0.295712,0.345742,0.323109,0.331149,0.244789,0.260274,0.241513,0.33383,0.322216,0.343955,0.195652,0.804348
std,2199.778908,14.732755,9.931512,14.859115,6.490198,2.84884,0.473606,0.467621,0.456430,0.475680,0.467734,0.470697,0.430026,0.438849,0.428064,0.47165,0.467395,0.475097,0.396761,0.396761
min,3.000000,18.000000,133.000000,20.700000,7.100000,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
25%,1831.250000,31.000000,158.200000,60.400000,21.700000,3.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1.000000
50%,3727.500000,43.000000,165.000000,70.200000,25.650000,6.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1.000000
75%,5663.750000,56.000000,171.600000,80.200000,30.000000,8.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.00000,1.000000,1.000000,0.000000,1.000000
max,7500.000000,69.000000,196.800000,123.800000,56.800000,10.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000


#Seprate output field from other fields

In [15]:
train_labels = train_dataset.pop('BMI')
test_labels = test_dataset.pop('BMI')

In [16]:
train_stats = train_dataset.describe()
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
ID,3358.0,3746.601251,2199.778908,3.0,1831.25,3727.5,5663.75,7500.0
Age,3358.0,43.716200,14.732755,18.0,31.00,43.0,56.00,69.0
Height_cm,3358.0,164.977725,9.931512,133.0,158.20,165.0,171.60,196.8
Weight_kg,3358.0,70.148243,14.859115,20.7,60.40,70.2,80.20,123.8
Sleep_Hours,3358.0,5.617630,2.848840,1.0,3.00,6.0,8.00,10.0
Gender_Male,3358.0,0.339488,0.473606,0.0,0.00,0.0,1.00,1.0
Gender_Other,3358.0,0.322811,0.467621,0.0,0.00,0.0,1.00,1.0
IsSmoker,3358.0,0.295712,0.456430,0.0,0.00,0.0,1.00,1.0
Alcohol_Low,3358.0,0.345742,0.475680,0.0,0.00,0.0,1.00,1.0
Alcohol_Moderate,3358.0,0.323109,0.467734,0.0,0.00,0.0,1.00,1.0


#Normilisation + Model Training

In [17]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

After the one-hot encoding, some original categorical columns might still be present in the `dataset` DataFrame. These columns are not numerical and will cause issues during normalization and model training if not removed. We need to explicitly drop them before splitting the data.

In [18]:
columns_to_drop = [
    'Gender', 'Smoker', 'Exercise_Freq', 'Diet_Quality', 'Alcohol_Consumption'
]
# Only drop columns that still exist in the dataset
dataset = dataset.drop(columns=[col for col in columns_to_drop if col in dataset.columns])
display(dataset.head())

,ID,Age,Height_cm,Weight_kg,BMI,Sleep_Hours,Gender_Male,Gender_Other,IsSmoker,Alcohol_Low,Alcohol_Moderate,Alcohol_High,Diet_Average,Diet_Good,Diet_Excellent,Exercise_1_2,Exercise_3_5,Exercise_Daily,SChronic_Disease_Yes,Chronic_Disease_No
1,2,69,169.3,70.7,24.7,2,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
2,3,46,159.1,69.0,27.3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
3,4,32,170.6,76.4,26.3,9,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,5,60,158.4,60.4,24.1,6,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
5,6,25,167.7,62.0,22.0,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0


In [19]:
model = build_model()
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         1,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,505 (21.50 KB)

 Trainable params: 5,505 (21.50 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
print("COLUMNS (in order):")
print(dataset.columns.to_list())
print("TOTAL FEATURES:", len(dataset.columns))

COLUMNS (in order):
['ID', 'Age', 'Height_cm', 'Weight_kg', 'BMI', 'Sleep_Hours', 'Gender_Male', 'Gender_Other', 'IsSmoker', 'Alcohol_Low', 'Alcohol_Moderate', 'Alcohol_High', 'Diet_Average', 'Diet_Good', 'Diet_Excellent', 'Exercise_1_2', 'Exercise_3_5', 'Exercise_Daily', 'SChronic_Disease_Yes', 'Chronic_Disease_No']
TOTAL FEATURES: 20


In [ ]:
EPOCHS = 1000
history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS)

Epoch 1/1000
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 467.1360 - mae: 19.9733 - mse: 467.1360
Epoch 2/1000
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 10.0393 - mae: 2.3944 - mse: 10.0393
Epoch 3/1000
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.6180 - mae: 1.2668 - mse: 2.6180
Epoch 4/1000
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.6402 - mae: 0.9986 - mse: 1.6402
Epoch 5/1000
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1649 - mae: 0.8361 - mse: 1.1649
Epoch 6/1000
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9951 - mae: 0.7654 - mse: 0.9951
Epoch 7/1000
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8578 - mae: 0.7050 - mse: 0.8578
Epoch 8/1000
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6940 - mae: 0.6363 - mse: 0.6940
Epoch 9/1000
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5760 - mae: 0.5888 - mse: 0.5760
Epoch 10/1000
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4738 - mae: 0.5292 - mse: 0.4738
Epoch 11/1000
105/105 

#Model Testing

In [ ]:
loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=0)
loss,mae,mse

(0.11590558290481567, 0.28196999430656433, 0.11590558290481567)

In [ ]:
test_predictions = model.predict(normed_test_data).flatten()
test_predictions

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


array([24.419542 , 23.578024 , 29.301432 , 29.965322 , 20.149527 ,
       22.27954  , 40.12396  , 29.582191 , 17.088522 , 16.984732 ,
       19.602505 , 22.080269 , 24.365768 , 21.0678   , 24.041838 ,
       24.65886  , 21.952885 , 36.418617 , 21.247847 , 18.455992 ,
       28.439814 , 38.482838 , 22.435286 , 25.120123 , 18.283876 ,
       24.389936 , 32.218193 , 41.03614  , 25.8053   , 23.939415 ,
       24.259474 , 26.887415 , 31.513683 , 24.237946 , 33.789005 ,
       23.898314 , 23.18408  , 27.120777 , 38.108284 , 30.9699   ,
       22.896729 , 41.817955 , 36.10411  , 21.215467 , 20.687943 ,
       24.290125 , 19.025385 , 17.357958 , 33.021896 , 23.671347 ,
       35.06733  , 21.892967 , 24.971712 , 15.419069 , 28.530394 ,
       25.279268 , 14.112259 , 19.14649  , 26.541895 , 16.845957 ,
       25.879728 , 18.71163  , 20.03192  , 24.747068 , 17.60501  ,
       30.684359 , 19.43957  , 30.205374 , 25.386608 , 18.975882 ,
       17.560549 , 28.90044  , 20.761917 , 13.700653 , 30.5111

In [ ]:
test_labels

,BMI
1,24.7
4,24.1
9,29.5
15,30.2
28,20.4
...,...
7461,24.8
7463,20.8
7473,18.0
7474,15.5


#Model Conversion


In [ ]:
kearas_file = "bodex.h5"
tf.keras.models.save_model(model,kearas_file)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tfmodel = converter.convert()
open("bodexmodel.tflite","wb").write(tfmodel)

Saved artifact at '/tmp/tmp0pubodv9'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 19), dtype=tf.float32, name='keras_tensor_8')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  138958866155280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138958866160656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138958866157776: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138958866157584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138958866161808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138958866161040: TensorSpec(shape=(), dtype=tf.resource, name=None)


23900

In [ ]:
from google.colab import files

# Download both files
files.download("bodex.h5")
files.download("bodexmodel.tflite")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
print(train_dataset.columns.to_list())
print("COUNT =", len(train_dataset.columns))


['ID', 'Age', 'Height_cm', 'Weight_kg', 'Sleep_Hours', 'Gender_Male', 'Gender_Other', 'IsSmoker', 'Alcohol_Low', 'Alcohol_Moderate', 'Alcohol_High', 'Diet_Average', 'Diet_Good', 'Diet_Excellent', 'Exercise_1_2', 'Exercise_3_5', 'Exercise_Daily', 'SChronic_Disease_Yes', 'Chronic_Disease_No']
COUNT = 19
